In [1]:
! wget https://storage.googleapis.com/kaggle-forum-message-attachments/237294/7771/german_credit_data.csv

/bin/sh: wget: command not found


In [2]:
import pandas as pd
df = pd.read_csv("german_credit_data.csv", index_col=0)

FileNotFoundError: [Errno 2] File b'german_credit_data.csv' does not exist: b'german_credit_data.csv'

In [ ]:
df = df.fillna(value="NA")

In [ ]:
df

In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer, make_column_transformer
preprocess = make_column_transformer(
    (['Age', 'Credit amount', 'Duration'], StandardScaler()),
    (['Sex', 'Job', 'Housing', 'Saving accounts', 'Checking account', 'Purpose', 'Risk'], OneHotEncoder(sparse=False))
)

In [ ]:
mat = preprocess.fit_transform(df)

In [ ]:
mat[10,:]

In [ ]:
X = mat[:, :-2]
Y = mat[:, -1]

In [ ]:
num_feats = X.shape[1]
numX = X.shape[0]

In [ ]:
import numpy as np

In [ ]:
datasize = 500
cs_size = 25
split_on_doc = 0.8
testsize = 100
ratio_relevant = 0.4
ratios_col = Y * ratio_relevant + (1-Y)*(1-ratio_relevant)

In [ ]:
# generate a candidate set of size 10 everytime
data_X = []
data_Y = []
test_X = []
test_Y = []
group_identities_train = []
group_identities_test = []
print("Sampling between 0 and {} for train".format(numX*split_on_doc))
p = ratios_col[0:int(numX*split_on_doc)]
p = p / sum(p)
for i in range(datasize):
    cs_indices = np.random.choice(np.arange(0, numX*split_on_doc, dtype=int), size=cs_size, p=p)
    cs_X = X[cs_indices]
    cs_Y = Y[cs_indices]
    data_X.append(cs_X)
    data_Y.append(cs_Y)
    group_identities_train.append(cs_X[:,4])
print("Sampling between {} and {} for test".format(numX*split_on_doc, numX))
p = ratios_col[int(numX*split_on_doc):]
p = p/sum(p)
for i in range(testsize):
    cs_indices = np.random.choice(np.arange(numX*split_on_doc, numX, dtype=int), size=cs_size, p=p)
    test_X.append(X[cs_indices])
    test_Y.append(Y[cs_indices])
    group_identities_test.append(X[cs_indices,4])

In [ ]:
import pickle as pkl
pkl.dump((data_X, data_Y), open("german_train_rank_3.pkl", "wb"))
pkl.dump((test_X, test_Y), open("german_test_rank_3.pkl", "wb"))
